In [18]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [19]:
ds = Dataset.load_from_disk("./nlpcc_2017/")
ds

Dataset({
    features: ['title', 'content'],
    num_rows: 5000
})

In [20]:
ds = ds.train_test_split(100, seed=42)
ds

Loading cached split indices for dataset at /mnt/workspace/nlpcc_2017/cache-1c09a3a414c6d4a6.arrow and /mnt/workspace/nlpcc_2017/cache-1de0c1dd37eb7170.arrow


DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 4900
    })
    test: Dataset({
        features: ['title', 'content'],
        num_rows: 100
    })
})

In [32]:
#加载模型
# 检查是否有可用的 GPU
path='./checkpoint-153'  #模型存在的文件夹

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained(path,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(path,trust_remote_code=True)
# 将模型移动到 GPU
model.to(device)


GLMForConditionalGeneration(
  (glm): GLMModel(
    (word_embeddings): VocabEmbedding()
    (transformer): GLMStack(
      (embedding_dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(1025, 1024)
      (block_position_embeddings): Embedding(1025, 1024)
      (layers): ModuleList(
        (0-23): 24 x GLMBlock(
          (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attention): SelfAttention(
            (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
            (attention_dropout): Dropout(p=0.1, inplace=False)
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (output_dropout): Dropout(p=0.1, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
            (dense_4h_to_h): Linear(in_fe

In [24]:
import torch

model = model.eval()

def predict_test():
    predict = []
    with torch.inference_mode():
        for d in ds["test"]:
            inputs = tokenizer("摘要生成: \n" + d["content"] + tokenizer.mask_token, return_tensors="pt")
            inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
            inputs = inputs.to("cuda")
            output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id, do_sample=True)
            predict.append(tokenizer.decode(output[0].tolist()).split("<|startofpiece|>")[1].replace("<|endofpiece|>", "").strip())
            print("curID:", len(predict))
    return predict

In [21]:
import re
import torch
import jieba.posseg as posseg
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

###################################################
# TextRank实现
###################################################
# 停用词路径
stopwords_path = './TextRank/stopwords.txt'
# 需要排除的词性
stopPOS = []

# 读取停用词
with open(stopwords_path, 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f.readlines()]

def segment_text_to_sentence(text):
    # 将文本分割成句子
    sentences = re.split(r'[。！？!?]', text)
    sentences = [sentence.strip().replace(" ", "").replace('\n','') for sentence in sentences if sentence.strip()]
    return sentences

def segment_text_to_words(text, use_stopwords):
    # 分词并去除停用词
    global stopPOS, stopwords
    stopPOS = [item.lower() for item in stopPOS]
    words = posseg.cut(text)
    if use_stopwords:
        words = [word for word, flag in words if flag[0].lower() not in stopPOS and word not in stopwords]
    else:
        words = [word for word, flag in words if flag[0].lower() not in stopPOS]
    words = set(words)

    return words

def original_similarity_matrix(sentences, use_stopwords):
    # 计算原始相似性矩阵
    sentence_words = [set(segment_text_to_words(item, use_stopwords)) for item in sentences]
    # print(sentence_words)
    size = len(sentences)
    similarity_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(i+1, size):
            if len(sentence_words[i]) == 0 or len(sentence_words[j]) == 0:
                similarity = 0
            else:
                # 计算相似性
                similarity = len(sentence_words[i] & sentence_words[j]) / (np.log(len(sentence_words[i])) + np.log(len(sentence_words[i])) + 1e-10)
            similarity_matrix[i][j] = similarity_matrix[j][i] = similarity
    return similarity_matrix

def cosine_tfidf_similarity_matrix(sentences, use_stopwords):
    # 计算基于TF-IDF的余弦相似性矩阵
    sentence_words = [' '.join(segment_text_to_words(item, use_stopwords)) for item in sentences]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentence_words)
    similarity_matrix = cosine_similarity(tfidf_matrix)

    # 将对角线元素设置为0，避免自身与自身的相似性干扰
    np.fill_diagonal(similarity_matrix, 0)
    return similarity_matrix

def summarize_text_rank(text, d=0.85, iter_num=200, top=3, method='默认方式', use_stopwords=True):
    sentences = segment_text_to_sentence(text)

    print('---------开始----------------------------------------')
    if method == '默认方式':
        edge_weight = original_similarity_matrix(sentences, use_stopwords)
    elif method == 'TF-IDF':
        edge_weight = cosine_tfidf_similarity_matrix(sentences, use_stopwords)

    node_weight = np.ones((len(sentences)))
    # print(node_weight)
    
#     print("句子间相似性计算(边的权重)")
#     print(edge_weight)

#     print("\n句子节点参数迭代更新：")
    for num in range(iter_num):                
        # TextRank迭代公式
        # print(node_weight)
        node_weight_new = (1-d) + d * node_weight @ (edge_weight / (edge_weight.sum(axis=-1) + 1e-10)).T
        if ((node_weight_new - node_weight)**2).sum() < 1e-10:
            break
        node_weight = node_weight_new
        # print(node_weight)

    if num < iter_num:
        print('迭代{}次，收敛'.format(num))
    else:
        print('迭代{}次，未收敛'.format(num))

    sorted_indices = np.argsort(node_weight)[::-1]

    # 获取最大的几个值及其对应的索引
    top_indices = sorted(sorted_indices[:top])
    top_values = node_weight[top_indices]

    print('最大的{}个值：'.format(top), top_values)
    print('对应的索引：', top_indices)
    print('结果：')
    result = ''
    for idx in top_indices:
        result += sentences[idx] + '。\n'
    print(result)

    return result

# # # 示例
# text = '记者7日从江苏省苏州市有关部门证实,该市一名初中副校长涉嫌泄露联考考题被停职,纪委已介入调查。今年4月,苏州市吴江区举行初三联考,震泽初级中学是联考学校之一,联考成绩将直接影响考生能否进入四星级高中(即重点高中)。就在考试前一天,考卷题目已在网上疯传,引发社会关注。据知情人士介绍,疑似震泽初级中学副校长将题目提前泄露给自己亲戚,然后这位亲戚又将题目进行了散播,导致题目最终被传到了网上。据苏州市吴江区教育局副局长沈正元介绍,泄题事件发生后,教育局于5月紧急推出了学校自主招生考试政策,以弥补泄题造成的不良后果。5月11日,教育局对涉嫌泄题违纪的震泽初级中学副校长作出停职处理,区纪委也已介入调查,教育部门会根据纪委处理意见进行相应处理。另据家长反映,泄题事件中一些“得题考生”出现在了震泽中学的保送名单里。对此,沈正元回应称,目前对涉嫌的涉事成年人进行了初步处理,纪委尚未作出明确处理意见,所以推优、自主招生等还按正常程序进行,因此出现了市民反映的疑似“得题考生”出现在名单中的情况。具体处理意见将在调查结果公布后作出,教育局将尽快进行相关调查工作,保障学生正当利益。(记者刘巍巍'
# summarize_text_rank(text)

In [22]:
import torch

def predict_test():
    predict = []
    with torch.inference_mode():
        for d in ds["test"]:
            output=summarize_text_rank(d["content"], d=0.85, iter_num=200, top=1, method='默认方式', use_stopwords=True)
            predict.append(output)
            print("curID:", len(predict))
    return predict

In [23]:
result = predict_test()

---------开始----------------------------------------
迭代8次，收敛
最大的1个值： [1.25248288]
对应的索引： [6]
结果：
英国《卫报》引述电脑安全专家亨特的话说,IS的说法或许危言耸听,因为该组织即便手中有资料,有可能也是早前因为各种不慎方式外泄的,未必是IS旗下黑客获得的,所以“有造假嫌疑”。

curID: 1
---------开始----------------------------------------
迭代8次，收敛
最大的1个值： [1.35559688]
对应的索引： [0]
结果：
中新网5月27日电据中共安徽省纪律检查委员会网站消息,日前,安徽宿松县招商局副局长李同保、交通运输局下属道路运输管理局副局长余刚涉嫌吸食毒品被开除党籍、行政撤职,宿松县纪委对县招商局、县交通运输局给予全县通报批评。

curID: 2
---------开始----------------------------------------
迭代10次，收敛
最大的1个值： [1.25691131]
对应的索引： [0]
结果：
新华网哈尔滨4月9日电(记者梁书斌)记者在黑龙江省公安厅了解到,非法生产、销售和使用“伪基站”设备违法犯罪活动日益突出。

curID: 3
---------开始----------------------------------------
迭代14次，收敛
最大的1个值： [1.24157269]
对应的索引： [0]
结果：
从明天开始,北京市将实行新一轮机动车尾号轮换。

curID: 4
---------开始----------------------------------------
迭代8次，收敛
最大的1个值： [1.2476539]
对应的索引： [0]
结果：
21日9时20分许,在哈市松北区松北一路55号一家宾馆门前人行道上,一名中年男子由于有外遇多天不回家,他的母亲与妻子找到他后,三人厮打在一起。

curID: 5
---------开始----------------------------------------
迭代10次，收敛
最大的1个值： [1.58356051]
对应的索引： [5]
结果：
5月11日,教育局对涉嫌泄题违纪的震泽

In [24]:
from rouge_chinese import Rouge

rouge = Rouge()

docode_preds = [" ".join(p) for p in result]
decode_labels = [" ".join(l) for l in ds["test"]["title"]]

scores = rouge.get_scores(docode_preds, decode_labels, avg=True)
{
    "rouge-1": scores["rouge-1"]["f"],
    "rouge-2": scores["rouge-2"]["f"],
    "rouge-l": scores["rouge-l"]["f"],
}

{'rouge-1': 0.37328073426616526,
 'rouge-2': 0.19245568474548322,
 'rouge-l': 0.2647670612629381}

In [25]:
from rouge_chinese import Rouge

rouge = Rouge()

docode_preds = [" ".join(p) for p in result]
decode_labels = [" ".join(l) for l in ds["test"]["title"]]

                
# 计算 ROUGE 分数
scores = rouge.get_scores(docode_preds, decode_labels, avg=True)

# 提取 ROUGE 的 Precision、Recall 和 F1 值
rouge_scores = {
    "rouge-1": {
        "precision": scores["rouge-1"]["p"],
        "recall": scores["rouge-1"]["r"],
        "f1": scores["rouge-1"]["f"]
    },
    "rouge-2": {
        "precision": scores["rouge-2"]["p"],
        "recall": scores["rouge-2"]["r"],
        "f1": scores["rouge-2"]["f"]
    },
    "rouge-l": {
        "precision": scores["rouge-l"]["p"],
        "recall": scores["rouge-l"]["r"],
        "f1": scores["rouge-l"]["f"]
    }
}
print(rouge_scores)

{'rouge-1': {'precision': 0.3244275550696497, 'recall': 0.47781737479872033, 'f1': 0.37328073426616526}, 'rouge-2': {'precision': 0.16283231279520569, 'recall': 0.2599414846079348, 'f1': 0.19245568474548322}, 'rouge-l': {'precision': 0.22318594032682315, 'recall': 0.36696002318302595, 'f1': 0.2647670612629381}}


In [26]:
from bert_score import BERTScorer
from rouge import Rouge
from transformers import BertTokenizer, BertForMaskedLM
from bert_score import BERTScorer
import jieba

model_name = 'bert-base-chinese'
tokenizer_A = BertTokenizer.from_pretrained(model_name)
model_A = BertForMaskedLM.from_pretrained(model_name)

# 初始化BERTScorer
scorer = BERTScorer(model_type=model_name, num_layers=model_A.config.num_hidden_layers)

# 存储每个样本的BERTScore结果
bert_scores = []
# ds["test"]["title"]
for pred, ref in zip(result, ds["test"]["title"]):
    P, R, F1 = scorer.score([pred], [ref])
    # 存储Precision、Recall和F1值
    bert_scores.append({
        "bert-precision": P.item(),
        "bert-recall": R.item(),
        "bert-f1": F1.item()
    })
# 计算平均BERTScore的Precision、Recall和F1值
avg_precision = sum(score["bert-precision"] for score in bert_scores) / len(bert_scores)
avg_recall = sum(score["bert-recall"] for score in bert_scores) / len(bert_scores)
avg_f1 = sum(score["bert-f1"] for score in bert_scores) / len(bert_scores)

print("\nAverage BERTScore:")
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1:", avg_f1)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Average BERTScore:
Average Precision: 0.6854258969426155
Average Recall: 0.7297082269191741
Average F1: 0.705667268037796


In [36]:
# input_text=''
import jieba
from rouge_chinese import Rouge

input_text = ds["test"][-1]["content"]
title = ds["test"][-1]["title"]

inputs = tokenizer("摘要生成: \n" + input_text + tokenizer.mask_token, return_tensors="pt")
inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
# inputs = inputs.to("cpu")
# 将输入张量移动到与模型相同的设备
inputs = {key: value.to(device) for key, value in inputs.items()}

output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id, do_sample=True)
print(tokenizer.decode(output[0].tolist()))

# 原始文本
text = tokenizer.decode(output[0].tolist())
# 提取数据
start_marker = '<|startofpiece|>'
end_marker = '<|endofpiece|>'
# 找到起始和结束位置
start_index = text.find(start_marker) + len(start_marker)
end_index = text.find(end_marker)
# 提取所需数据
extracted_data = text[start_index:end_index].strip()
# 打印结果

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[CLS] 摘要生成: 中国证券网讯(记者严政)中国重工6月14日晚间公告称,公司日前接到控股股东中国船舶重工集团公司(简称“中船重工”)通知,中船重工拟对其自身相关业务进行调整,部分业务涉及到公司。其中,公司拟以持有的动力相关资产进行对外投资,参与中船重工拟打造的动力业务平台公司。公司上述对外投资的方案不构成重大资产重组,也不涉及公司发行股份。中国重工表示,目前方案还需进一步论证,存在不确定性。为保证公平信息披露,维护投资者利益,避免造成公司股价异常波动,经公司申请,自6月12日下午开市起公司股票停牌。同时公司将与中船重工保持密切联系,尽快确认是否进行上述事项,并于股票停牌之日起5个工作日内(含停牌当日)公告事项进展情况。[MASK]<|endoftext|> <|startofpiece|> 中国重工拟斥资14亿参建动力业务平台,包括参与中船重工打造的动力业务平台公司;或涉及公司公开增发等事项。 <|endofpiece|>


In [41]:
from rouge import Rouge
from transformers import BertTokenizer, BertForMaskedLM
from bert_score import BERTScorer
import jieba

# 定义预测和实际值
result = title
predict = extracted_data

# 加载BERT模型和tokenizer
model_name = './bert-base-chinese'
tokenizer_A = BertTokenizer.from_pretrained(model_name)
model_A = BertForMaskedLM.from_pretrained(model_name)
# 计算BERTScore
scorer = BERTScorer(model_type=model_name, num_layers=model_A.config.num_hidden_layers, batch_size=1)
P, R, F1 = scorer.score([predict], [result])

# 初始化 ROUGE 评分器
rouge = Rouge()
# 使用 jieba 进行分词
result_tokens = " ".join(jieba.cut(result))  # 实际值分词
predict_tokens = " ".join(jieba.cut(predict))  # 预测值分词


test=ds["test"][-1]["content"]

print('\n原文Content内容为:  '+test)


print('\n原文title内容为:  '+title)
print("\n")
print('文本摘要内容为:  '+extracted_data)
print("\n分词情况\n")
print(result_tokens)
print(predict_tokens)

# 将实际值和预测值放入列表
decode_preds = [predict_tokens]  # 预测值
decode_labels = [result_tokens]  # 实际值

# 计算 ROUGE 分数
scores = rouge.get_scores(decode_preds, decode_labels, avg=True)

# 提取 ROUGE 分数
rouge_scores = {
    "rouge-1": scores['rouge-1']['f'],
    "rouge-2": scores['rouge-2']['f'],
    "rouge-l": scores['rouge-l']['f'],
}

# 输出 ROUGE 和 BERTScore 结果
print("\nROUGE 分数:")
print(rouge_scores)

print("\nBERTScore:")
bert_scores = {
    "bert-precision": P.item(),
    "bert-recall": R.item(),
    "bert-f1": F1.item()
}
# 输出转换后的 BERTScore 结果
print(bert_scores)

Some weights of the model checkpoint at ./bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



原文Content内容为:  中国证券网讯(记者严政)中国重工6月14日晚间公告称,公司日前接到控股股东中国船舶重工集团公司(简称“中船重工”)通知,中船重工拟对其自身相关业务进行调整,部分业务涉及到公司。其中,公司拟以持有的动力相关资产进行对外投资,参与中船重工拟打造的动力业务平台公司。公司上述对外投资的方案不构成重大资产重组,也不涉及公司发行股份。中国重工表示,目前方案还需进一步论证,存在不确定性。为保证公平信息披露,维护投资者利益,避免造成公司股价异常波动,经公司申请,自6月12日下午开市起公司股票停牌。同时公司将与中船重工保持密切联系,尽快确认是否进行上述事项,并于股票停牌之日起5个工作日内(含停牌当日)公告事项进展情况。

原文title内容为:  中国重工拟以持有的动力相关资产进行对外投资,参与中船重工拟打造的动力业务平台公司,将继续停牌。


文本摘要内容为:  中国重工拟斥资14亿参建动力业务平台,包括参与中船重工打造的动力业务平台公司;或涉及公司公开增发等事项。

分词情况

中国 重工 拟以 持有 的 动力 相关 资产 进行 对外 投资 , 参与 中 船 重工 拟 打造 的 动力 业务 平台 公司 , 将 继续 停牌 。
中国 重工 拟 斥资 14 亿 参建 动力 业务 平台 , 包括 参与 中 船 重工 打造 的 动力 业务 平台 公司 ; 或 涉及 公司 公开 增发 等 事项 。

ROUGE 分数:
{'rouge-1': 0.559999995008, 'rouge-2': 0.3703703653772291, 'rouge-l': 0.5084745712841139}

BERTScore:
{'bert-precision': 0.8306502103805542, 'bert-recall': 0.8300982713699341, 'bert-f1': 0.8303741812705994}


Bert值计算

In [12]:
def generate_summary(input_text, tokenizer=tokenizer, model=model, device=device):
    # 生成摘要的输入
    inputs = tokenizer("摘要生成: \n" + input_text + tokenizer.mask_token, return_tensors="pt")
    inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)

    # 将输入张量移动到与模型相同的设备
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # 生成输出
    output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id, do_sample=True)

    # 解码生成的文本
    text = tokenizer.decode(output[0].tolist())
    # print(text)

    # 提取数据
    start_marker = '<|startofpiece|>'
    end_marker = '<|endofpiece|>'
    # 找到起始和结束位置
    start_index = text.find(start_marker) + len(start_marker)
    end_index = text.find(end_marker)
    # 提取所需数据
    extracted_data = text[start_index:end_index].strip()

    return extracted_data

# input_text = ds["test"][5]["content"]
# generate_summary(input_text)


input_text = ds["test"][5]["content"]
print('原文内容： '+input_text)
summary = generate_summary(input_text, tokenizer, model, device)
print('\n文本摘要内容为: ' + summary)
print('文本标题内容: ' + ds["test"][5]["title"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


原文内容： 记者7日从江苏省苏州市有关部门证实,该市一名初中副校长涉嫌泄露联考考题被停职,纪委已介入调查。今年4月,苏州市吴江区举行初三联考,震泽初级中学是联考学校之一,联考成绩将直接影响考生能否进入四星级高中(即重点高中)。就在考试前一天,考卷题目已在网上疯传,引发社会关注。据知情人士介绍,疑似震泽初级中学副校长将题目提前泄露给自己亲戚,然后这位亲戚又将题目进行了散播,导致题目最终被传到了网上。据苏州市吴江区教育局副局长沈正元介绍,泄题事件发生后,教育局于5月紧急推出了学校自主招生考试政策,以弥补泄题造成的不良后果。5月11日,教育局对涉嫌泄题违纪的震泽初级中学副校长作出停职处理,区纪委也已介入调查,教育部门会根据纪委处理意见进行相应处理。另据家长反映,泄题事件中一些“得题考生”出现在了震泽中学的保送名单里。对此,沈正元回应称,目前对涉嫌的涉事成年人进行了初步处理,纪委尚未作出明确处理意见,所以推优、自主招生等还按正常程序进行,因此出现了市民反映的疑似“得题考生”出现在名单中的情况。具体处理意见将在调查结果公布后作出,教育局将尽快进行相关调查工作,保障学生正当利益。(记者刘巍巍)

文本摘要内容为: 苏州吴江区一初中副校长泄考题,遭停职审查;学生因“得题”而获保送资格
文本标题内容: 苏州一初中副校长涉嫌泄露联考题目给亲戚,导致题目被传到网上,副校长停职被纪委调查;联考成绩直接影响考生能否进重点高中。


In [10]:
from ipywidgets import Button, Textarea, Output, HBox, VBox

import string
# 数字转换为汉字的字典
num_to_chinese = {
    "0": "零",
    "1": "一",
    "2": "二",
    "3": "三",
    "4": "四",
    "5": "五",
    "6": "六",
    "7": "七",
    "8": "八",
    "9": "九",
    "10": "十",
    "11": "十一",
    "12": "十二"
}
def remove_all_punctuation_and_convert_number_to_chinese(text):
    # 创建一个翻译表，用于将所有标点符号替换为空格
    translator = str.maketrans('', '', string.punctuation + "。"+"、"+"'"+"“"+"”"+"："+":"+" "+","+"，")
    # 使用翻译表去除文本中的所有标点符号
    text_without_punctuation = text.translate(translator)
    
    # 将文本中的数字转换为汉字
    for digit, chinese_character in num_to_chinese.items():
        text_without_punctuation = text_without_punctuation.replace(digit, chinese_character)
    
    return text_without_punctuation


import re
def contains_non_chinese_text(input_text):
    # 使用正则表达式匹配非中文字符
    non_chinese_pattern = re.compile(r'[^\u4e00-\u9fff]')  # 匹配不是中文字符的正则表达式
    match = non_chinese_pattern.search(input_text)

    if match:
        return True
    else:
        return False
def check_text_input(input_text):
    if len(input_text)<10:
        return 0
    
#     return 2

#     if not isinstance(input_text, str):
#         return 0
    if contains_non_chinese_text(input_text):
        return 1
    else:
        return 2
# # 示例用法
# check_text_input('这是一段文本内容')
# check_text_input('Hello World')
# check_text_input('1234')


# 定义一个简单的函数
def greet(b):
    input_text = remove_all_punctuation_and_convert_number_to_chinese(input_widget.value)
    output_num=check_text_input(input_text)  #数字输出 判断输入文本的类型
    if input_widget.value == '':
        text = '内容为空，请输入文本内容。'
    else:
        if output_num == 0:
            text = '输入不是文本，请输入文本内容。'
        elif output_num==1:
            text = '输入包含非中文字符，请输入中文文本。'
            # print( '输入包含非中文字符，请输入中文文本。')
            # text = generate_summary(input_widget.value)
        else:
            text = generate_summary(input_widget.value)
    
    output_widget.clear_output()
    with output_widget:
        print(f'{text}')

# 创建一个输出控件
output_widget = Output()

# 创建一个大的文本输入控件
input_widget = Textarea(
    placeholder='输入你的文本...',
    description='文本输入:',
    layout={'width': '600px', 'height': '300px'},  # 设置宽度和高度
    style={'overflow': 'auto'}  # 允许滚动
)

# 创建一个按钮
submit_button = Button(description="生成摘要")

# 将按钮的点击事件与 greet 函数关联
submit_button.on_click(greet)
# 增加间隔
# input_widget.layout.margin = '30px 30px 30px 30px'  # 上右下左的间距
# submit_button.layout.margin = '0 0 30px 0'  # 上右下左的间距
# 布局
ui = VBox([input_widget, submit_button, output_widget])

# 显示控件
ui